In [ ]:
from flask import Flask, render_template, redirect, url_for, flash, send_file
from flask_wtf import FlaskForm
from wtforms import FileField, SubmitField
from wtforms import StringField
from wtforms.validators import DataRequired
import ipfshttpclient
import os 
import time 
import torch
from transformers import pipeline
#from hugfacefunc import func
from transformers import pipeline
pipe = pipeline("text2text-generation", model="Shxck69/DimSum")


D:\Anaconda\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sahil\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'

# Function to save file to IPFS
#import os
file_mapping={}
def save_to_ipfs(file):
    try:
        # Get the file name
        file_name = file.filename

        # Connect to the IPFS daemon
        client = ipfshttpclient.connect('/ip4/127.0.0.1/tcp/5001/http')

        # Save the file to a temporary location
        #temp_path = os.path.join('/tmp', file_name)
        #file.save(temp_path)
        # Open and read the file content
        with open(file_name, 'rb') as file:
            res = client.add(file)
            return res['Hash']
        # Add the file to IPFS
        #res = client.add(temp_path)

        # Remove the temporary file
        #os.remove(temp_path)
        file_mapping[file_name]=res['Hash']
        return res['Hash']
    except Exception as e:
        print(f"Error saving to IPFS: {str(e)}")
        return None

# Function to retrieve file from IPFS
def retrieve_from_ipfs(ipfs_hash):
    try:
        client = ipfshttpclient.connect('/ip4/127.0.0.1/tcp/5001/http')
        file_data = client.cat(ipfs_hash)
        return file_data
    except Exception as e:
        print(f"Error retrieving from IPFS: {str(e)}")
        return None

# File Upload Form
class UploadForm(FlaskForm):
    file = FileField('File')
    submit = SubmitField('Upload')

# File upload route
@app.route('/upload', methods=['GET', 'POST'])
def upload():
    form = UploadForm()
    if form.validate_on_submit():
        # Save the file to IPFS
        ipfs_hash = save_to_ipfs(form.file.data)
        if ipfs_hash:
            flash(f'File uploaded to IPFS with hash: {ipfs_hash}', 'success')
        else:
            flash('Error uploading file to IPFS', 'error')
        return redirect(url_for('upload'))
    return render_template('upload.html', form=form)

# File download route
@app.route('/download/<ipfs_hash>')
def download(ipfs_hash):
    # Retrieve the file from IPFS
    file_data = retrieve_from_ipfs(ipfs_hash)
    if file_data:
        return send_file(file_data, as_attachment=True)
    else:
        flash('Error retrieving file from IPFS', 'error')
        return redirect(url_for('index'))

# Index route
@app.route('/')
def index():
    return render_template('index.html')

def narrative(ind,st):
    try:
        l = len(st) // 3
        h = str(st)
        p1 = h[ind:l]
        p2 = h[l:l+l]
        p3 = h[l+l:]
        #print(len(p1),len(p2),len(p3))
        s1 = pipe(p1, max_length=70, min_length=20, do_sample=False)
        s2  = pipe(p2,max_length=70,min_length=20,do_sample=False)
        s3  = pipe(p3,max_length=70,min_length=20,do_sample=False)
            #s4  = summarizer(p4,max_length=35,min_length=20,do_sample=False)
        return (s1[0]['generated_text']+s2[0]['generated_text']+s3[0]['generated_text'])
    except:
        narrative(ind+75,st)
   
    
# Function for HuggingFace processing (replace with actual processing logic)
def huggingface_processing(ipfs_hash):
    # Use a pipeline as a high-level helper
    #from transformers import pipeline
    #pipe = pipeline("text2text-generation", model="Shxck69/DimSum")
    f = retrieve_from_ipfs(ipfs_hash)
    st=str(f)
    if(len(st) > 3500):
        out = narrative(400,st)
    else:
        out=pipe(st,max_length=100,min_length=2,do_sample=False)
    #print(len(f))
    # Placeholder: You should implement the actual HuggingFace processing logic here
    # For example, you can simply return a modified version of the file data
    #return f'HuggingFace processed content for IPFS Hash: {ipfs_hash}'
    return out

# Process Form
class ProcessForm(FlaskForm):
    ipfs_hash = StringField('IPFS Hash', validators=[DataRequired()])
    submit = SubmitField('Process')

# Process route
@app.route('/process', methods=['GET', 'POST'])
def process():
    form = ProcessForm()
    if form.validate_on_submit():
        # Get the IPFS hash from the form
        ipfs_hash = form.ipfs_hash.data

        # Perform HuggingFace processing (replace with actual processing logic)
        processed_data = huggingface_processing(ipfs_hash)

        if processed_data:
            flash(f'HuggingFace processing complete for IPFS Hash: {ipfs_hash}', 'success')
            # You can display or redirect to a new page showing the processed data
            return render_template('processed_data.html', processed_data=processed_data)
        else:
            flash('Error during HuggingFace processing', 'error')
    return render_template('process.html', form=form)

# File Download Form
class DownloadForm(FlaskForm):
    file_name = StringField('File Name', validators=[DataRequired()])
    submit = SubmitField('Download')

# File download route
@app.route('/download', methods=['GET', 'POST'])
def download_file():
    form = DownloadForm()
    if form.validate_on_submit():
        # Get the file name from the form
        file_name = form.file_name.data

        # Perform a lookup to retrieve the IPFS hash for the given file name
        ipfs_hash = lookup_ipfs_hash_by_name(file_name)

        if ipfs_hash:
            return redirect(url_for('download', ipfs_hash=ipfs_hash))
        else:
            flash(f'File with name {file_name} not found', 'error')

    return render_template('download.html', form=form)

def lookup_ipfs_hash_by_name(file_name):
    return file_mapping.get(file_name)

if __name__ == '__main__':
    app.run(debug=False)


In [ ]:
    <form method="get" action="{{ url_for('download_link') }}">
        <div>
            <label for="file_name">Enter File Name:</label>
            <input type="text" id="file_name" name="file_name" required>
        </div>
        <div>
            <button type="submit">Download</button>
        </div>
    </form>